# Imports

In [1]:
"""
Created on 	: 12/10/19
Developer 	: Yeshwanth Reddy
File Type	: Python
"""

import pandas as pd
import numpy as np
import heapq


# Data Pre Processing
Before Starting the project, we have the car dataset with all the "d features of each car". Now, we need to generate the customers data based on these features.<br>
Along with all the feature details, we also know the class of a car (unacc, acc, good, vgood). So, we can use this data to create satisfaction bit stings for each car assuming we have 575 customers.<br>
I have used a random function and have chosen if a customer is satisfied by a product or not. If the car is of class vgood, then the random function gives 1 as output for 90% of times.<br>
vgood - 90%<br>
good - 75%<br>
acc - 50%<br>
unacc - 10%<br>

Using this technique, I have constructed a SBS table of dimensions 1728x575. The rows are the products and columns are the customers. It has been saved to a SBS.csv file in data_process folder.

In [2]:
#    buyingCost   v-high, high, med, low
#    maintCost    v-high, high, med, low
#    doors        2, 3, 4, 5-more
#    seats        2, 4, more
#    lug_boot     small, med, big
#    safety       low, med, high

def create_SBS(fileLocation = "car/car.data"):
	fileName = fileLocation.split("/")[-1].split(".")[0]
	print("Reading data...")
	data = pd.read_csv(fileLocation, header=None)
	data = np.array(data)

	print("Formatting data...")
	data[data=='5more'] = 5
	data[data=='vhigh'] = 3
	data[data=='more'] = 2
	data[data=='high'] = 2
	data[data=='big'] = 2
	data[data=='med'] = 1
	data[data=='low'] = 0
	data[data=='small'] = 0
	# Classes defining if a user BUYs or NOT.
	data[data=='unacc'] = 0
	data[data=='acc'] = 1
	data[data=='good'] = 2
	data[data=='vgood'] = 3
	
	data = np.array(data, dtype=np.uint8)

	print("creating Empty SBS table...")
	no_cust = data.shape[0]//3
	SBS = np.zeros((data.shape[0], no_cust), dtype=np.uint8)
	
	print("Filling SBS table...")
	for i in range(data.shape[0]):
		for j in range(SBS.shape[1]):
			if data[i][-1] == 3: 	# 90% chance
				SBS[i][j]= 0 if random.randint(0,100)%10==0 else 1
			elif data[i][-1] == 2:  # 75% chance
				SBS[i][j]= 0 if random.randint(0,100)%4==0 else 1
			elif data[i][-1] == 1:  # 50% chance
				SBS[i][j]= 0 if random.randint(0,100)%2==0 else 1
			elif data[i][-1] == 0:  # 10% chance
				SBS[i][j]= 1 if random.randint(0,100)%10==0 else 0
	
	print("Saving to file...")
	pd.DataFrame(data, dtype=np.uint8).to_csv("data_process/"+fileName+".csv", header=None, index=None)
	pd.DataFrame(SBS, dtype=np.uint8).to_csv("data_process/"+fileName+"_SBS.csv", header=None, index=None)
	return SBS

def read_SBS(fileLocation):
	SBS = pd.read_csv(fileLocation, header=None)
	return SBS


# KMDP 

There are 2 things.
1. Predicting the satisfaction bit string(SBS) of a candidate product.
2. Predicting the k-most demanding products based on SBS.

Formulas and Descriptions-<br>
$C$ = {$c_0, c_1, c_2,....c_nc$}  -- Customers<br>
$EP$ = {$ep_0, ep_1, ep_2,....ep_nep$}  -- Existing products<br>
$CP$ = {$cp_0, cp_1, cp_2,....cp_ncp$}  -- Candidate products<br>
$kCP$ = {$kCP_1, kCP_2, kCP_3,....kCP_n$}  --  Set of Clusters<br>
$kCP_1$ = {$cp_0, cp_3, cp_5,....cp_9$}  --  Cluster of candidate products<br><br>
$p$($cp_i, c_j$) = $\frac{1}{N(EP, c)+N(CP, c)}$ -- if $c_j$ likes $cp_i$<br>
$p$($cp_i, c_j$) = 0  --  otherwise<br><br>
E($kCP, C$) = $\sum_{i=1}^{kcp} \sum_{j=1}^{C} p(cp_i, c_j)$ 


In [3]:
def N_Calc():
	Nsbs = [[0]*2]*w
	for i in range(w):
		EP_count=0
		CP_count=0
		for j in range(0, (h*30)//100):
			if SBS[j][i]==1:
				EP_count+=1
		for j in range((h*30)//100, h):
			if SBS[j][i]==1:
				CP_count+=1
		Nsbs[i][0] = EP_count
		Nsbs[i][1] = CP_count
	return Nsbs

def N(cust, Nsbs):
	EP_count = Nsbs[cust][0]
	CP_count = Nsbs[cust][1]
	return EP_count+CP_count

def P(cp_i, c_j, Nsbs):
	P=0
	if SBS[cp_i, c_j]==1:
		P = 1 / N(c_j, Nsbs)
	else:
		pass
	return P

def E(kCP, C, Nsbs):
	E = 0
	for cp_i in kCP:
		for c_j in C:
			E += P(cp_i, c_j, Nsbs)
	return E


## Single product based Greedy algorithm

Let S denote a set containing a single candidate product cp in CP. <br>
The SPG algorithm uses E(S, C), which computes the expected number of the customers in C for S, as the ranking function of the candidate products. <br><br>
S = {$cp_0$}<br>
$p$($S, c_j$) = $\frac{1}{N(EP, c_j)+N(CP, c_j)}$ -- if $c_j$ likes S<br>
$p$($S, c_j$) = 0  -- otherwise<br><br>
E($S, C$) = $\sum_{j=1}^{C} p(S, c_j)$ 

The candidate products with the top-k values of the ranking function are selected to form an approximate solution of the k-MDP discovering problem.<br>
Algorithm 1: The SPG Algorithm <br>
Input: N_vector(EP,C), set C of customer requirements, set CP
of candidate product, and the value k <br>
Output: A set of k candidate product
```
i=0;
for each candidate product cp in CP {
    compute the satisfaction bit string of cp;
    S={cp};
    Compute E(S,C);
    i=i+1;
    if i<=k
        Insert E(S,C) into the top-k list
    elif E(S,C)> the smallest value in top-k list
        Replace the smallest in top-k list with E(S,C);
    }
    Put the corresponding candidate products in the top-k list to set the kCP;
return kCP;
```

In [15]:
def Espga(S, C, Nsbs):
	E = 0
	for cj in C:
		E+=P(S, cj, Nsbs)
	return E

def SPGA(SBS, k):
	k = [(0, 0)]*k
	heapq.heapify(k)
	Nsbs = N_Calc()
	for cp in CP:
		S = cp
		E = Espga(S, C, Nsbs)
		heapq.heappush(k, (E, cp))
		heapq.heappop(k)
	return k


In [24]:
# Creating the Satisfaction Bit String of every customer
try:
	SBS = read_SBS("data_process/car_SBS.csv")
except:
	SBS = create_SBS("car/car.data")
SBS = np.array(SBS, dtype=np.uint8)
np.random.shuffle(SBS)

h=len(SBS)
w=len(SBS[0])

# Creating Sets of Customers, Candidate Products and Existing Products.
C = np.arange(0, w)
EP = np.arange(0, (h*30)//100)
CP = np.arange((h*30)//100, h)
# kCP = ?

# Taking K as 5
k=38
k = SPGA(SBS, k)

print("Single Product Based Greedy Algorithm : \n", k)


Single Product Based Greedy Algorithm : 
 [(1.157175398633264, 1624), (1.1594533029612824, 922), (1.1594533029612824, 1173), (1.1594533029612824, 1436), (1.1594533029612824, 1510), (1.1731207289293923, 1635), (1.164009111617319, 1098), (1.1617312072893007, 999), (1.164009111617319, 1723), (1.1799544419134473, 1167), (1.170842824601374, 1573), (1.1753986332574107, 1438), (1.177676537585429, 1077), (1.1799544419134473, 1597), (1.1662870159453373, 1136), (1.1685649202733557, 1357), (1.184510250569484, 1381), (1.170842824601374, 1009), (1.1662870159453373, 1631), (1.191343963553539, 539), (1.1799544419134473, 1323), (1.1799544419134473, 1716), (1.1890660592255207, 660), (1.1890660592255207, 1177), (1.1799544419134473, 1472), (1.198177676537594, 989), (1.1890660592255207, 628), (1.1822323462414657, 1594), (1.1958997722095757, 1199), (1.1890660592255207, 1259), (1.1867881548975023, 728), (1.1753986332574107, 959), (1.177676537585429, 1165), (1.2072892938496673, 1296), (1.191343963553539, 155

## Markdown references
https://csrgxtu.github.io/2015/03/20/Writing-Mathematic-Fomulars-in-Markdown/


#### Greek Letters
$\alpha$ <br>
$A$ 	 <br>
$\beta$  <br>
$B$ 	 <br>
$\gamma$ <br>
$\Gamma$ <br>
$\pi$ 	 <br>
$\Pi$ 	 <br>
$\phi$ 	 <br>
$\Phi$ 	 <br>
$\varphi$ <br>
$\theta$  <br>

#### Operators
$\cos$  <br>
$\sin$ 	<br>
$\lim$ 	<br>
$\exp$  <br>
$\to$ 	<br>
$\infty$ 	<br>
$\equiv$ 	<br>
$\bmod$ 	<br>
$\times$ 	<br>

#### Powers and Indices
$k_{n+1}$ <br>
$n^2$ 	<br>
$k_n^2$ <br>

#### Fractions and Binomials
$\frac{n!}{k!(n-k)!}$ <br>
$\binom{n}{k}$ 	<br>
$\frac{\frac{x}{1}}{x - y}$ 	<br>
$^3/_7$ 	<br>

#### Roots
$\sqrt{k}$ 	<br>
$\sqrt[n]{k}$ <br>

#### Sums and Integrals
$\sum_{i=1}^{10} t_i$ 	<br>
$\int_0^\infty \mathrm{e}^{-x}\,\mathrm{d}x$ 	<br>
$\sum$ 	<br>
$\prod$ 	<br>
$\coprod$ 	<br>
$\bigoplus$ 	<br>
$\bigotimes$ 	<br>
$\bigodot$ 	<br>
$\bigcup$ 	<br>
$\bigcap$ 	<br>
$\biguplus$ 	<br>
$\bigsqcup$ 	<br>
$\bigvee$ 	<br>
$\bigwedge$ 	<br>
$\int$ 	<br>
$\oint$ 	<br>
$\iint$ 	<br>
$\iiint$ 	<br>
$\idotsint$ 	<br>
$\sum_{\substack{0<i<m\0<j<n}} P(i, j)$ 	<br>
$\int\limits_a^b$ 	<br>
$a’$ $a^{\prime}$ 	<br>
$a’’$ 	<br>
$\hat{a}$ 	<br>
$\bar{a}$ 	<br>
$\grave{a}$ 	<br>
$\acute{a}$ 	<br>
$\dot{a}$   <br>
$\ddot{a}$ 	<br>
$\not{a}$ 	<br>
$\mathring{a}$ 	<br>
$\overrightarrow{AB}$ 	<br>
$\overleftarrow{AB}$ 	<br>
$a’’’$ 	<br>
$\overline{aaa}$ 	<br>
$\check{a}$ 	<br>
$\vec{a}$ 	<br>
$\underline{a}$ 	<br>
$\color{red}x$ 	<br>
$\pm$ 	<br>
$\mp$ 	<br>
$\int y \mathrm{d}x$ 	<br>
$\,$ 	<br>
$\:$ 	<br>
$\;$ 	<br>
$!$ 	<br>
$\int y\, \mathrm{d}x$ 	<br>
$\dots$ 	<br>
$\ldots$ 	<br>
$\cdots$ 	<br>
$\vdots$ 	<br>
$\ddots$ 	<br>

#### Brackets
$(a)$ 	<br>
$[a]$ 	<br>
${a}$ 	<br>
$\langle f \rangle$ 	<br>
$\lfloor f \rfloor$ 	<br>
$\lceil f \rceil$ 	<br>
$\ulcorner f \urcorner$ 	<br>